In [1]:
# Mixed Precision Training

In [2]:
!wget https://raw.githubusercontent.com/kurumbus/tensorflow-training/refs/heads/master/helper_functions.py

--2025-04-18 07:08:36--  https://raw.githubusercontent.com/kurumbus/tensorflow-training/refs/heads/master/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py.1’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0.001s  

2025-04-18 07:08:36 (18.0 MB/s) - ‘helper_functions.py.1’ saved [10246/10246]



In [4]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys